In [ ]:
from GoogleNews import GoogleNews
from datetime import datetime

# Set topic, date range, and language
topic = "bitcoin"
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 3)
language = "en"

googlenews = GoogleNews(lang=language)
googlenews.set_time_range(
    start_date.strftime("%m/%d/%Y"), end_date.strftime("%m/%d/%Y")
)

# Retrieve news
news_results = googlenews.search(topic)

# Process news results (titles, links, etc.)
news_results